Aplicacion del Algoritmo BFGS

------------------------------------------------------------------------------

In [ ]:
import numpy as np

# ---------------------------------------------------------
# Definición de la función, gradiente y log-sum-exp seguro
# ---------------------------------------------------------
def f(x):
    # x es un vector [x0, x1]
    x1, x2 = x
    A = np.exp(x1**2 + x2**2) + 10 * np.exp(x1)
    return np.log(A)

def grad_f(x):
    x1, x2 = x
    A = np.exp(x1**2 + x2**2) + 10 * np.exp(x1)
    dfdx1 = (2 * x1 * np.exp(x1**2 + x2**2) + 10 * np.exp(x1)) / A
    dfdx2 = (2 * x2 * np.exp(x1**2 + x2**2)) / A
    return np.array([dfdx1, dfdx2])

# ---------------------------------------------------------
# Búsqueda de línea con condición de Armijo
# ---------------------------------------------------------
def line_search_armijo(f, grad_f, xk, pk, alpha0=1.0, c=1e-4, rho=0.5):
    alpha = alpha0
    fk = f(xk)
    gradk = grad_f(xk)
    while f(xk + alpha * pk) > fk + c * alpha * np.dot(gradk, pk):
        alpha *= rho
        if alpha < 1e-10:
            break
    return alpha

# ---------------------------------------------------------
# Método BFGS
# ---------------------------------------------------------
def bfgs(f, grad_f, x0, tol=1e-6, max_iter=1000):
    xk = x0.copy()
    n = len(xk)
    Hk = np.eye(n)  # matriz de aproximación del inverso del Hessiano
    fk = f(xk)
    gk = grad_f(xk)
    iter_data = [(0, xk.copy(), fk, np.linalg.norm(gk))]

    for k in range(1, max_iter + 1):
        # Dirección de búsqueda
        pk = -Hk.dot(gk)

        # Búsqueda de línea (Armijo)
        alpha = line_search_armijo(f, grad_f, xk, pk)

        # Actualización de x
        x_new = xk + alpha * pk
        g_new = grad_f(x_new)
        s = x_new - xk
        y = g_new - gk

        # Condición de actualización BFGS (evitar divisiones malas)
        if np.dot(y, s) > 1e-10:
            rho = 1.0 / np.dot(y, s)
            I = np.eye(n)
            Hk = (I - rho * np.outer(s, y)) @ Hk @ (I - rho * np.outer(y, s)) + rho * np.outer(s, s)

        # Actualizar variables
        xk, gk, fk = x_new, g_new, f(x_new)
        iter_data.append((k, xk.copy(), fk, np.linalg.norm(gk)))

        # Criterios de paro
        if np.linalg.norm(gk) < tol:
            break

    return xk, fk, np.linalg.norm(gk), k, iter_data

# ---------------------------------------------------------
# Ejemplo de ejecución
# ---------------------------------------------------------
if __name__ == "__main__":
    x0 = np.array([0.0, 0.0])   # punto inicial
    x_opt, f_opt, grad_norm, iters, history = bfgs(f, grad_f, x0)

    print("Resultado BFGS:")
    print(f"  x* = {x_opt}")
    print(f"  f(x*) = {f_opt:.6f}")
    print(f"  ||grad|| = {grad_norm:.2e}")
    print(f"  Iteraciones = {iters}")


El script está organizado en tres bloques funcionales:

Definición del problema: f(x) y grad_f(x).

Búsqueda de línea: line_search_armijo(...).

Algoritmo BFGS: bfgs(...) que llama a la búsqueda de línea y actualiza la aproximación del inverso del Hessiano.
Al final hay un if __name__ == "__main__": con un ejemplo de ejecución.

1. Explicacion de la funcion :

Qué hace: grad_f(x) Calcula ∇f usando las fórmulas analíticas que derivamos.

Por qué usar el gradiente analítico: BFGS es un método cuasi-Newton que requiere gradientes para construir las actualizaciones. Calcular el gradiente analíticamente es más preciso y rápido que aproximarlo por diferencias finitas.


2. Explicacion de la funcion :

line_search_armijo(...) — búsqueda de línea simple (Armijo / backtracking)

Qué hace: 
- Intenta alpha=1 y si el nuevo punto no satisface la condición de Armijo (descenso suficiente), reduce alpha multiplicándolo por rho (0.5) repetidamente (backtracking).

Parámetros importantes:

c: constante de Armijo (típico 10^(-4))

rho: factor de reducción (0.5) — cada iteración divide alpha por 2.

Por qué usar Armijo simple: es fácil de implementar y suele ser suficiente para BFGS en problemas suaves.

3. Explicacion de la funcion :

bfgs(f, grad_f, x0, tol=1e-6, max_iter=1000): - El nucleo del algoritmo

Inicialización

Hk = I: asumimos inicialmente que el inverso del Hessiano es identidad (solución neutra).

gk = grad_f(xk): empezamos con el gradiente en el punto inicial.

iter_data: se guarda historia para análisis o trazados.

Dirección de búsqueda pk

pk = -Hk.dot(gk) produce una dirección que incorpora información aproximada de curvatura; si Hk=I, pk es la dirección de descenso por gradiente.

Búsqueda de línea

Se usa la función Armijo descrita. El tamaño alpha determina cuánto avanzar.

Actualizar s y y

s = x_{k+1} - x_k y y = g_{k+1} - g_k son los vectores que BFGS usa para actualizar la matriz inversa del Hessiano.

Condición y^T s > 0

El requisito teórico para BFGS es 𝑦^(⊤) * s>0 (curvatura positiva). Si no se cumple, la fórmula estándar puede introducir indefinidades o dividir por cero.

En el código se exige > 1e-10 (pequeña tolerancia) para evitar divisiones numéricas malas; si no se cumple, se omite la actualización (o en implementaciones avanzadas se modifica y o se reinicia Hk).

Esta salvaguarda es práctica y frecuente.

Fórmula de actualización

La actualización implementada es la forma clásica del BFGS para la aproximación del inverso del Hessiano:

$$
    \mathbf{H}_{k+1} = \left(\mathbf{I} - \rho_k \mathbf{s}_k \mathbf{y}_k^T\right)\mathbf{H}_k\left(\mathbf{I} - \rho_k \mathbf{y}_k \mathbf{s}_k^T\right) + \rho_k \mathbf{s}_k \mathbf{s}_k^T
$$
Donde el escalar $\rho_k$ se define como:
$$
    \rho_k = \frac{1}{\mathbf{y}_k^T \mathbf{s}_k}
$$

Es la forma que mantiene simetría y (si se cumple la condición de curvatura) positividad definida de 𝐻𝑘+1

Criterios de paro

np.linalg.norm(gk) < tol: norma del gradiente por debajo del umbral → detenido.

-------------------------------------------------------------------------------

Algoritmo de Newton amortiguado (Damped Newton / Trust-Region)

------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
import numpy as np

# -----------------------------
# Evaluación numéricamente estable de f, gradiente y Hessiano
# f(x,y) = ln(e^{x^2+y^2} + 10 e^{x})
# Usamos log-sum-exp para evitar overflow.
# -----------------------------
def f_stable(x):
    x1, x2 = float(x[0]), float(x[1])
    a = x1**2 + x2**2
    b = x1 + np.log(10.0)    # log(10 * e^{x1}) = x1 + ln(10)
    M = max(a, b)
    val = M + np.log(np.exp(a - M) + np.exp(b - M))
    return val

def grad_f_stable(x):
    x1, x2 = float(x[0]), float(x[1])
    a = x1**2 + x2**2
    b = x1 + np.log(10.0)
    M = max(a, b)
    ea = np.exp(a - M)
    eb = np.exp(b - M)
    denom = ea + eb
    # derivadas en escala estable
    dfdx1 = (2 * x1 * ea + eb) / denom
    dfdx2 = (2 * x2 * ea) / denom
    return np.array([dfdx1, dfdx2])

def hess_f_stable(x):
    # Calculamos el Hessiano de f en forma estable (2x2)
    x1, x2 = float(x[0]), float(x[1])
    a = x1**2 + x2**2
    b = x1 + np.log(10.0)
    M = max(a, b)
    ea = np.exp(a - M)
    eb = np.exp(b - M)
    denom = ea + eb

    # Componentes auxiliares (numeradores escalados)
    A1 = 2 * x1 * ea         # proviene de 2 x1 e^a (escalado)
    B1 = eb                  # proviene de 10 e^{x1} (escalado)
    num_g1 = A1 + B1         # numerador de g1 (componente x1 del gradiente)
    # g2 numerador
    num_g2 = 2 * x2 * ea

    # derivadas de A1, B1
    # dA1/dx1 = 2*ea + 2*x1 * d(ea)/dx1, y d(ea)/dx1 = 2 x1 ea
    dA1_dx1 = 2 * ea + 2 * x1 * (2 * x1 * ea)   # 2 ea + 4 x1^2 ea
    dA1_dx2 = 2 * x1 * (2 * x2 * ea)             # 4 x1 x2 ea
    dB1_dx1 = eb
    dB1_dx2 = 0.0

    # derivadas del denom
    ddenom_dx1 = (2 * x1 * ea) + eb
    ddenom_dx2 = (2 * x2 * ea)

    # H_11: d(g1)/dx1 por regla del cociente
    H11 = ((dA1_dx1 + dB1_dx1) * denom - num_g1 * ddenom_dx1) / (denom**2)

    # H_12: d(g1)/dx2
    H12 = ((dA1_dx2 + dB1_dx2) * denom - num_g1 * ddenom_dx2) / (denom**2)

    # H_21: d(g2)/dx1
    # g2 = (2 x2 ea)/denom -> derivada wrt x1:
    # numerator derivative: 2 x2 * d(ea)/dx1 = 2 x2 * (2 x1 ea)
    dnum_g2_dx1 = 2 * x2 * (2 * x1 * ea)
    H21 = (dnum_g2_dx1 * denom - num_g2 * ddenom_dx1) / (denom**2)

    # H_22: d(g2)/dx2
    # derivative numerator: 2 ea + 2 x2 * d(ea)/dx2 = 2 ea + 4 x2^2 ea
    dnum_g2_dx2 = 2 * ea + 4 * x2**2 * ea
    H22 = (dnum_g2_dx2 * denom - num_g2 * ddenom_dx2) / (denom**2)

    H = np.array([[H11, H12],
                  [H21, H22]])
    # forzamos simetría numérica
    H = 0.5 * (H + H.T)
    return H

# -----------------------------
# Backtracking Armijo line search
# -----------------------------
def backtracking_armijo(f, grad, xk, pk, alpha0=1.0, c=1e-4, rho=0.5, max_iters=50):
    alpha = alpha0
    fk = f(xk)
    gk = grad(xk)
    for _ in range(max_iters):
        newx = xk + alpha * pk
        if f(newx) <= fk + c * alpha * np.dot(gk, pk):
            return alpha
        alpha *= rho
    return alpha

# -----------------------------
# Damped Newton / Trust-region style algorithm
# (Levenberg-Marquardt style damping + Armijo backtracking)
# -----------------------------
def damped_newton_trust(f, grad, hess, x0, tol=1e-8, max_iter=200,
                        lambda0=1e-3, lambda_factor_increase=10.0, lambda_factor_decrease=0.1):
    xk = x0.astype(float).copy()
    lamb = lambda0
    history = []
    for k in range(1, max_iter + 1):
        fk = f(xk)
        gk = grad(xk)
        grad_norm = np.linalg.norm(gk)
        history.append((k, xk.copy(), fk, grad_norm, lamb))
        if grad_norm < tol:
            break

        Hk = hess(xk)
        success = False

        # intentos con damping creciente para asegurar PD y dirección de descenso
        for attempt in range(20):
            try:
                H_reg = Hk + lamb * np.eye(len(xk))
                pk = np.linalg.solve(H_reg, -gk)
            except np.linalg.LinAlgError:
                lamb *= lambda_factor_increase
                continue

            # si no es dirección de descenso, aumentamos damping
            if np.dot(gk, pk) >= 0:
                lamb *= lambda_factor_increase
                continue

            # búsqueda de línea Armijo para el paso propuesto
            alpha = backtracking_armijo(f, grad, xk, pk, alpha0=1.0)
            newx = xk + alpha * pk
            newf = f(newx)

            # criterio simple: aceptamos si hay descenso (podemos refinar con ratio tipo trust-region)
            if newf <= fk + 1e-12:
                success = True
                break
            else:
                lamb *= lambda_factor_increase

        if not success:
            # fallback: gradiente con paso pequeño si no se encuentra paso Newton estable
            pk = -gk
            alpha = backtracking_armijo(f, grad, xk, pk, alpha0=1e-3)
            xk = xk + alpha * pk
            lamb *= lambda_factor_increase
            continue

        # actualizar x y reducir damping (recuperar comportamiento Newton)
        xk = newx
        lamb = max(1e-16, lamb * lambda_factor_decrease)

    # valores finales
    fk = f(xk)
    gk = grad(xk)
    history.append(("final", xk.copy(), fk, np.linalg.norm(gk), lamb))
    return xk, fk, np.linalg.norm(gk), k, history

# -----------------------------
# Ejemplo de ejecución (main)
# -----------------------------
if __name__ == "__main__":
    initials = [
        np.array([0.0, 0.0]),
        np.array([1.0, 0.0]),
        np.array([-1.0, 2.0]),
        np.array([2.0, 2.0])
    ]

    results = []
    for x0 in initials:
        xopt, fopt, gnorm, its, hist = damped_newton_trust(
            f_stable, grad_f_stable, hess_f_stable, x0
        )
        print("Init:", x0, "-> x*:", np.round(xopt, 6),
              " f*:", np.round(fopt, 8), "||grad||:", np.format_float_scientific(gnorm, 2),
              "iter:", its)
        results.append((x0, xopt, fopt, gnorm, its))

    # Graficar convergencia (norma del gradiente) del último historial como ejemplo
    try:
        import matplotlib.pyplot as plt
        hist = hist  # historial de la última ejecución
        iter_nums = [h[0] for h in hist if isinstance(h[0], int)]
        grad_norms = [h[3] for h in hist if isinstance(h[0], int)]
        if len(iter_nums) > 0:
            plt.semilogy(iter_nums, grad_norms, marker='o')
            plt.xlabel('Iteración')
            plt.ylabel('||grad|| (escala log)')
            plt.title('Convergencia (ejemplo)')
            plt.grid(True)
            plt.show()
    except Exception:
        # si no hay matplotlib, no graficamos pero el algoritmo sigue funcionando
        pass


El objetivo es minimizar la función:$$f(x,y) = \ln(e^{x^2+y^2} + 10 e^{x})$$Cuando los exponentes $x^2+y^2$ o $x$ son muy grandes, $e^{\text{exponente}}$ puede causar un overflow (desbordamiento) en la computadora. Para evitar esto, se utiliza la técnica Log-Sum-Exp (logaritmo de la suma de exponenciales), que reescribe $\ln(e^A + e^B)$ como:$$\ln(e^A + e^B) = \max(A, B) + \ln(e^{A-\max(A, B)} + e^{B-\max(A, B)})$$Esta forma es numéricamente estable.

1. Función $f(x)$ (f_stable)
- x1, x2 = float(x[0]), float(x[1]): Extrae las componentes $x_1$ y $x_2$ del vector de entrada $x$.

- a = $x1^2 + x2^2$: Corresponde al exponente del primer término: $A = x^2+y^2$.

- b = x1 + np.log(10.0): Corresponde al exponente del segundo término: $B = x + \ln(10)$, ya que $10 e^x = e^{\ln(10)} e^x = e^{x + \ln(10)}$.

- M = max(a, b): Calcula el máximo $M = \max(A, B)$.

- val = M + np.log(np.exp(a - M) + np.exp(b - M)): Implementa la fórmula estable de Log-Sum-Exp.

-return val: Devuelve el valor de la función.

2. Gradiente de $f(x)$ (grad_f_stable)

Calcula el vector de las primeras derivadas parciales $\nabla f(x) = (\frac{\partial f}{\partial x_1}, \frac{\partial f}{\partial x_2})^T$, utilizando la derivada de la forma estable:$$\frac{\partial f}{\partial x_i} = \frac{1}{e^{A} + e^{B}} \cdot (\frac{\partial e^{A}}{\partial x_i} + \frac{\partial e^{B}}{\partial x_i})$$Al escalar por $e^{-M}$ arriba y abajo:$$\frac{\partial f}{\partial x_i} = \frac{e^{-M}}{e^{-M}} \frac{e^{-M} (\frac{\partial e^{A}}{\partial x_i} + \frac{\partial e^{B}}{\partial x_i})}{e^{-M} (e^{A} + e^{B})} = \frac{\frac{\partial e^{A}}{\partial x_i} e^{-M} + \frac{\partial e^{B}}{\partial x_i} e^{-M}}{e^{A-M} + e^{B-M}}$$

- ea = np.exp(a - M) y eb = np.exp(b - M): Son los términos escalados $e^{A-M}$ y $e^{B-M}$.

- denom = ea + eb: Es el denominador común escalado.
- dfdx1 = (2 * x1 * ea + eb) / denom: $\frac{\partial f}{\partial x_1}$. Los numeradores son las derivadas $\frac{\partial A}{\partial x_1} e^{A-M} = 2x_1 e^{A-M}$ y $\frac{\partial B}{\partial x_1} e^{B-M} = 1 \cdot e^{B-M}$.
- dfdx2 = (2 * x2 * ea) / denom: $\frac{\partial f}{\partial x_2}$. Los numeradores son $\frac{\partial A}{\partial x_2} e^{A-M} = 2x_2 e^{A-M}$ y $\frac{\partial B}{\partial x_2} e^{B-M} = 0 \cdot e^{B-M}$.
- return np.array([dfdx1, dfdx2]): Devuelve el gradiente.

3. Hessiano de $f(x)$ (hess_f_stable)

Calcula la matriz de las segundas derivadas parciales $\mathbf{H}(x)$, $\mathbf{H}_{ij} = \frac{\partial^2 f}{\partial x_i \partial x_j}$. Utiliza la regla del cociente para derivar las componentes del gradiente (que ya están en forma estable).
- Calcula términos auxiliares, incluyendo las derivadas de los numeradores y el denominador escalados, para luego aplicar la regla del cociente: $\frac{d}{dx} \left(\frac{u}{v}\right) = \frac{u'v - uv'}{v^2}$.
- H11, H12, H21, H22: Son las cuatro componentes del Hessiano.
- H = np.array([[H11, H12], [H21, H22]]): Ensambla la matriz Hessiana.
- H = 0.5 * (H + H.T): Fuerza la simetría numérica del Hessiano, lo cual es teórico para funciones suaves (Teorema de Clairaut).
- return H: Devuelve la matriz Hessiana.

4. Búsqueda de Línea Armijo (backtracking_armijo)
Esta función se utiliza para encontrar un tamaño de paso ($\alpha$) que asegure un descenso suficiente en el valor de la función a lo largo de la dirección de búsqueda ($p_k$).
- Entradas: La función $f$, el gradiente grad, el punto actual $x_k$, y la dirección de búsqueda $p_k$.
- Criterio de Armijo: Busca un $\alpha$ (empezando por alpha0=1.0) tal que:$$f(x_k + \alpha p_k) \le f(x_k) + c \cdot \alpha \cdot \nabla f(x_k)^T p_k$$Donde $c$ (c=1e-4) es un factor de descenso pequeño.
- Mecanismo: Si el criterio no se cumple, el paso $\alpha$ se reduce multiplicándolo por $\rho$ (rho=0.5) (proceso de backtracking o retroceso), y se repite hasta que el criterio se cumpla o se alcance max_iters.
- return alpha: Devuelve el paso $\alpha$ encontrado.

5. Newton Amortiguado (damped_newton_trust)

Implementa el método de Newton amortiguado, que es una mezcla entre el método de Newton y el método de Máximo Descenso. Se utiliza un término de amortiguamiento (damping) $\lambda$ (similar al algoritmo de Levenberg-Marquardt o un enfoque tipo Trust-Region) para asegurar que la dirección de búsqueda sea una dirección de descenso.
- Bucle Principal: Se repite hasta que la norma del gradiente (grad_norm) sea menor que la tolerancia tol o se alcance max_iter.
- Cálculo de la Dirección de Newton Amortiguada:
   - H_reg = Hk + lamb * np.eye(len(xk)): Se añade el término de amortiguamiento $\lambda$ a la diagonal del Hessiano $H_k$, creando una matriz regularizada $H_{reg}$. Esto asegura que $H_{reg}$ sea Definida Positiva (o al menos mejor condicionada) y la dirección $p_k$ calculada sea una dirección de descenso.
   - pk = np.linalg.solve(H_reg, -gk): Resuelve el sistema lineal para encontrar la dirección de Newton amortiguada $p_k$.
- Manejo de Fallos (Damping Adjustment):
   - Si la solución falla (LinAlgError), o si la dirección no es de descenso (np.dot(gk, pk) >= 0), $\lambda$ se aumenta (lambda *= lambda_factor_increase), y se intenta de nuevo (aumentando el damping).
- Búsqueda de Línea y Actualización:
   - Si se encuentra una dirección de descenso, se usa backtracking_armijo para encontrar el tamaño de paso $\alpha$.
   - newx = xk + alpha * pk: Se calcula el nuevo punto.
   - Si el nuevo punto reduce el valor de la función (newf <= fk + 1e-12), el paso es exitoso (success = True):
       - xk = newx: Se acepta el nuevo punto.
       - lamb se reduce (lamb = max(1e-16, lamb * lambda_factor_decrease)) para recuperar el comportamiento de Newton puro (convergencia cuadrática).
   - Si no hay descenso, $\lambda$ se aumenta nuevamente.
- Fallback (Retroceso): Si el bucle de intentos (attempt in range(20)) falla en encontrar un paso Newton aceptable, se utiliza un paso de máximo descenso (pk = -gk) con un $\alpha$ muy pequeño.
- return xk, fk, np.linalg.norm(gk), k, history: Devuelve el punto óptimo, el valor de la función, la norma final del gradiente, el número de iteraciones y un historial de convergencia.